# Prediction Delivery Time

# EDA Dataset

## Explore

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   

In [ ]:
list1 =np.array(range(10))
list1

In [20]:
df_ebay = pd.read_csv('../data/raw/ebayShort.csv', index_col=0)

As the data is too big, we first working in the shortcut of the total data with 1 million rows.

In [ ]:
# Look at the first 5 row

df_ebay.head()

In [21]:
#Look at 5 random row
df_ebay.sample(5)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
177376,C2C,3427893,3.0,2018-03-15 16:31:00.000-04:00,0,25.00,3,5,02170,56572,6,862.00,1,2018-03-14 07:50:23.000-04:00,2018-03-17,8,1,PACKAGE_THICK_ENVELOPE,177377
471991,B2C,125374,NaN,2018-10-29 11:49:00.000-07:00,1,16.45,2,5,80528,35216,0,39.99,1,2018-10-26 16:12:17.000-06:00,2018-10-31,53,1,PACKAGE_THICK_ENVELOPE,471992
810623,C2C,452556,3.0,2019-03-04 13:25:00.000-08:00,0,4.81,3,5,93510,21811,8,77.99,1,2019-03-01 18:01:15.000-08:00,2019-03-09,8,1,PACKAGE_THICK_ENVELOPE,810624
558285,B2C,194,1.0,2018-04-18 19:02:00.000-07:00,0,0.00,3,5,518000,20783,5,5.99,1,2018-04-17 05:47:04.000-07:00,2018-04-21,0,1,LETTER,558286
718184,C2C,1069328,1.0,2018-04-13 12:23:00.000-04:00,4,7.59,2,9,08052,33002,4,5.51,1,2018-04-02 20:02:30.000-04:00,2018-04-17,8,1,PACKAGE_THICK_ENVELOPE,718185


In [ ]:
#check the shape of the dataset
df_ebay.shape

In [ ]:
print(f'There are {df_ebay.shape[0]} columns and {df_ebay.shape[1]} rows')

In [ ]:
# Get a quick overview of dataset variables
df_ebay.info()

In [ ]:
# top 5 rows showing only 'object' columns
df_ebay.select_dtypes('object').head()

In [ ]:
plt.figure(figsize=(14, 26))
plt.subplots(2, 2)

plt.subplot(2, 2, 1)
plt.pie(df_ebay['b2c_c2c'].value_counts(), labels= df_ebay['b2c_c2c'].unique(), labeldistance=0.5, autopct='%1.1f%%')
plt.title('portion of B2C and C2C')

plt.subplot(2, 2, 2)
plt.scatter( df_ebay['item_price'], df_ebay['carrier_max_estimate'])

plt.show()

## Data Cleaning

First let do some common checking in the dataset.
 -  Null value
 -  Duplicate row

In [34]:
#Checking null
df_ebay.isna().sum()

b2c_c2c                          0
seller_id                        0
declared_handling_days       47180
acceptance_scan_timestamp        0
shipment_method_id               0
shipping_fee                     0
carrier_min_estimate             0
carrier_max_estimate             0
item_zip                         0
buyer_zip                        0
category_id                      0
item_price                       0
quantity                         0
payment_datetime                 0
delivery_date                    0
weight                           0
weight_units                     0
package_size                     0
record_number                    0
dtype: int64

There is 47180 missing value in total 1 million rows. We could delete it but I decide to have deepper look at it later


In [ ]:
# checking duplicate rows
df_ebay.duplicated().any()

There are no duplicate row in the database

In [ ]:
#recall shape column
df_ebay.shape[1]

Let check out the distributions, we are going to loop over the columns and print some subplot.

In [ ]:
#checking the b2c_c2c column
df_ebay['b2c_c2c'].value_counts()

Convert `b2c_c2c` column to binary

In [ ]:
df_ebay['b2c_c2c']= np.where(df_ebay['b2c_c2c']=='B2C', 1, 0)

Test it

In [ ]:
df_ebay.head()

There is 2 `weight_units` is 1 and 2. Let say 1 - 's' and 2 - 'kg'. We are going to convert all the weight units to lb.

In [22]:
#check the record has weigtht unit =2 to able to compare
df_ebay[df_ebay['weight_units']==2].head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
115821,B2C,46815,1.0,2018-01-29 10:06:00.000-06:00,0,0.0,3,5,35503,30605,1,6.89,2,2018-01-28 20:18:59.000-06:00,2018-01-31,113,2,LARGE_ENVELOPE,115822
847218,B2C,40624,1.0,2019-10-03 13:18:00.000-05:00,2,0.0,2,9,63044,85715,6,26.99,1,2019-10-03 01:28:56.000-05:00,2019-10-10,0,2,NONE,847219
908707,C2C,295500,1.0,2018-03-21 14:00:00.000-04:00,5,7.0,2,5,02050,02109,1,29.30,1,2018-03-04 08:51:33.000-05:00,2018-03-07,13,2,LARGE_ENVELOPE,908708


In [23]:
df_ebay['weight'] = np.where(df_ebay['weight_units']==1, df_ebay['weight'], df_ebay['weight']*2.20462 )

check the record with `weight_units`=2 to see if the `weight` column is change

In [24]:
df_ebay[df_ebay['weight_units']==2]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
115821,B2C,46815,1.0,2018-01-29 10:06:00.000-06:00,0,0.0,3,5,35503,30605,1,6.89,2,2018-01-28 20:18:59.000-06:00,2018-01-31,249.12206,2,LARGE_ENVELOPE,115822
847218,B2C,40624,1.0,2019-10-03 13:18:00.000-05:00,2,0.0,2,9,63044,85715,6,26.99,1,2019-10-03 01:28:56.000-05:00,2019-10-10,0.00000,2,NONE,847219
908707,C2C,295500,1.0,2018-03-21 14:00:00.000-04:00,5,7.0,2,5,02050,02109,1,29.30,1,2018-03-04 08:51:33.000-05:00,2018-03-07,28.66006,2,LARGE_ENVELOPE,908708


Now  we change the `weight_units` column all to the same unit

In [25]:
df_ebay['weight_units']= np.where(df_ebay['weight_units']== 2, 1, 1)


In [28]:
#check
df_ebay[df_ebay['weight_units']==2].value_counts()

Series([], dtype: int64)